In [1]:
import time
import copy
import random

import torch
import torch.nn as nn

import torchvision
import numpy as np

from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

In [2]:
epochs = 10
batch = 16
lr = 0.001
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])

train_dataset = torchvision.datasets.MNIST('./mnist', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST('./mnist', train=False, download=True,transform=transform)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch, num_workers=4, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch, num_workers=4, shuffle=True)

In [4]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        
        self.fc1 = nn.Linear(64, 64)
        self.fc2 = nn.Linear(64, 10)
        
        self.act = nn.ReLU()
        self.pool = nn.AdaptiveMaxPool2d(output_size=(1, 1))
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.act(x)
        
        x = self.conv2(x)
        x = self.act(x)
        
        x = self.pool(x)
        x = torch.flatten(x, 1)
        
        x = self.fc1(x)
        x = self.act(x)
        
        x = self.fc2(x)
        return x

In [5]:
def eval(model, dataloader):
    avg_accuracy = 0
    avg_f1 = 0
    
    model.eval()
    for imgs, labels in tqdm(dataloader):
        imgs = imgs.to(device)

        with torch.no_grad():
            out = model(imgs)

        out = torch.argmax(out, dim=1)
        avg_accuracy += accuracy_score(labels.numpy(), out.detach().numpy())
        avg_f1 += f1_score(labels.numpy(), out.detach().numpy(), average='macro')

    avg_accuracy /= len(test_dataloader)
    avg_f1 /= len(test_dataloader)

    print(f'Accuracy : {round(avg_accuracy, 3)}')
    print(f'F1 : {round(avg_f1, 3)}')

In [6]:
def train(model, optimizer, criteria, dataloader, epochs):
    for e in range(epochs):
        desc = f'Epoch: {e+1}/{epochs}'
        avg_loss = 0
        
        model.train()
        for imgs, labels in tqdm(dataloader, desc=desc): 
            imgs = imgs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad() 
            output = model(imgs)

            loss = criteria(output, labels)
            avg_loss += loss.item()

            loss.backward()
            optimizer.step()

        avg_loss /= len(train_dataloader)
        print('Loss', round(avg_loss, 3))

In [7]:
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criteria = torch.nn.CrossEntropyLoss()
    
train(model, optimizer, criteria, train_dataloader, 1)
eval(model, test_dataloader)

  0%|          | 0/625 [00:00<?, ?it/s]

Loss 0.261


100%|██████████| 625/625 [00:13<00:00, 47.72it/s]

Accuracy : 0.976
F1 : 0.961


In [21]:
i = random.randint(0, len(test_dataset))
print('Item', i)

img, label = test_dataset[i]
out = model(img.unsqueeze(0).to(device))
out = torch.argmax(out, dim=1)

print(f'Predict : {out.item()}. Real : {label}')

torchvision.transforms.ToPILImage()(img)

Item 9553
Predict : 9. Real : 9
